In [86]:
import numpy as np
import torch

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

epochs = 10
learning_rate= 0.01
momentum=0.5

In [87]:
# Values for Normalize() are the global mean and
# standard deviation of the MNIST dataset
transform = transforms.Compose([lambda img: transforms.functional.rotate(img, -90, fill=(0,)),
                                lambda img: transforms.functional.hflip(img),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.EMNIST('dataset/', split='balanced', train=True,
                             download=True, transform=transform)

test_data = datasets.EMNIST('dataset/', split='balanced', train=False,
                            download=True, transform=transform)

In [95]:
train_loader = DataLoader(train_data,
                          batch_size=64,
                          shuffle=True)

test_loader = DataLoader(test_data,
                         batch_size=1000,
                         shuffle=True)
print(len(train_loader.dataset))

112800


In [89]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.dropout = nn.Dropout()  # Dropout to prevent overfitting
        self.fc1 = nn.Linear(7 * 7 * 256, 512)
        self.fc2 = nn.Linear(512, 64)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.dropout(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.softmax(out)

        return out

In [90]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
cnn = Net()
cnn.to(device)

Net(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=12544, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (softmax): LogSoftmax()
)

In [91]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=learning_rate, momentum=momentum)
def train(epoch_n):
    running_loss = 0
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        output = cnn(images)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if idx % 100 == 0:
            print('Epoch {} -> [{}/{} ({:.0f}%)] Loss: {:.6}'.format(
                epoch_n, idx * len(images), len(train_loader.dataset),
                100. * idx / len(train_loader), running_loss/len(train_loader))
            )
        

In [92]:
def test():
    correct, total, test_loss = 0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            output = cnn(images)
            
            predicted_label = output.data.max(1, keepdim=True)[1]
            correct += predicted_label.eq(labels.to(device).
                                          data.view_as(predicted_label)).sum()
        
            loss = criterion(output, labels)
            test_loss += loss.item()
        
        test_loss /= len(test_loader.dataset)   
        print('Accuracy = {:.2f}%'.format(100. * correct/len(train_loader.dataset)))
        

In [93]:
test()
for e in range(1, epochs + 1):
    train(e)
    test()

Accuracy = 15.60%
Epoch 1 -> [0/112800 (0%)] Loss: 0.00236055


KeyboardInterrupt: 

In [ ]:
def view_prediction():
    images, labels = next(iter(test_loader))
    # Turn off gradients to speed up this part
    with torch.no_grad():
        output = cnn(images.to(device))
    # Convert the log-probability for checking probability of each class
    probability = torch.exp(output)
    prediction = list(output.cpu().numpy()[0])
    prediction.
   
